In [2]:
# set the environment variables needed for openai package to know to reach out to azure
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://<your-endpoint.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "your AzureOpenAI key"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

In [3]:
text = "This is a test document."

query_result = embeddings.embed_query(text)

In [4]:
doc_result = embeddings.embed_documents([text, text, text])

In [6]:
from langchain.vectorstores.redis import Redis
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./contract.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=20)
docs = text_splitter.split_documents(documents)



In [13]:
rds = Redis.from_documents(
    docs,
    embeddings,
    redis_url="redis://localhost:6379",
    index_name="link",
)

In [7]:
rds.similarity_search("在什么情况下可以单方面解除合同")

[Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除的； 2．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作，甲方单方面解除本合同的； \n3．乙方年度考核或者聘期考核不合格， 又不同意甲方调整其工作岗位的，', metadata={'source': './contract.pdf', 'page': 7}),
 Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除的； 2．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作，甲方单方面解除本合同的； \n3．乙方年度考核或者聘期考核不合格， 又不同意甲方调整其工作岗位的，', metadata={'source': './contract.pdf', 'page': 7}),
 Document(page_content='（二）乙方有下列情形之一的，甲方可以随时单方面解除本合同： 1．在试用期内被证明不符合本岗位要求； 2．连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的； 3．乙方公派出国或因私出境逾期不归；   \n4．违反工作规定或者操作规程，发生责 任事故，或者失职、渎职，造成严\n重后果的； \n5．严重扰

In [8]:
retriever = rds.as_retriever()

In [14]:
retriever.get_relevant_documents("在什么情况下可以单方面解除合同")

[Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除的； 2．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作，甲方单方面解除本合同的； \n3．乙方年度考核或者聘期考核不合格， 又不同意甲方调整其工作岗位的，', metadata={'source': './contract.pdf', 'page': 7}),
 Document(page_content='（二）乙方有下列情形之一的，甲方可以随时单方面解除本合同： 1．在试用期内被证明不符合本岗位要求； 2．连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的； 3．乙方公派出国或因私出境逾期不归；   \n4．违反工作规定或者操作规程，发生责 任事故，或者失职、渎职，造成严\n重后果的； \n5．严重扰乱工作秩序，致使甲方、其他单位工作不能正常进行的； 6．被判处拘役、有期徒刑缓刑以及有期 徒刑以上刑罚收监执行，或者被劳\n动教养的。 \n（三）乙方有下列情形之一，甲方可以单方面解除本合同，但是应当提前\n30 日以书面形式通知乙方： \n1．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作的；', metadata={'source': './contract.pdf', 'page': 6}),
 Document(page_content='83．因工负伤，治疗终结后经劳动能力鉴定机构鉴定为 1 至 4 级丧失劳动能\n力的； \n4．患职业病以及现有医疗条件下难以治愈的严重疾病或者精神病的； 5．乙方正在接受纪律审查尚未作出结论的； 6．属于国家规定的不得解除本合同的其他情形的。 （五）有下列情形之一的，乙方可以随时单方面解除本合同： 1．在试用期内的； 

In [17]:
retriever = rds.as_retriever(search_type="similarity_limit", k=2, score_threshold=0.8)

In [18]:
retriever.get_relevant_documents("在什么情况下可以单方面解除合同")

[Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除的； 2．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作，甲方单方面解除本合同的； \n3．乙方年度考核或者聘期考核不合格， 又不同意甲方调整其工作岗位的，', metadata={'source': './contract.pdf', 'page': 7}),
 Document(page_content='（二）乙方有下列情形之一的，甲方可以随时单方面解除本合同： 1．在试用期内被证明不符合本岗位要求； 2．连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的； 3．乙方公派出国或因私出境逾期不归；   \n4．违反工作规定或者操作规程，发生责 任事故，或者失职、渎职，造成严\n重后果的； \n5．严重扰乱工作秩序，致使甲方、其他单位工作不能正常进行的； 6．被判处拘役、有期徒刑缓刑以及有期 徒刑以上刑罚收监执行，或者被劳\n动教养的。 \n（三）乙方有下列情形之一，甲方可以单方面解除本合同，但是应当提前\n30 日以书面形式通知乙方： \n1．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作的；', metadata={'source': './contract.pdf', 'page': 6})]

In [4]:
from langchain.chat_models import AzureChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0)

In [ ]:
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

In [17]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [19]:
unique_docs = retriever_from_llm.get_relevant_documents("在什么情况下可以单方面解除合同")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 什么条件下可以单方面终止合同？', '2. 哪些情况下可以单方面解除合同？', '3. 在哪些情况下可以单方面取消合同？']


2

Chain

In [16]:
rds = Redis.from_existing_index(
    embeddings, redis_url="redis://localhost:6379", index_name="link"
)
retriever = rds.as_retriever()

In [7]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)

In [9]:
print(qa.run("在什么情况下可以单方面解除合同"))

根据所提供的文本，有以下情况可以单方面解除合同：

- 甲方因工负伤，治疗终结后经劳动能力鉴定机构鉴定为 1 至 4 级丧失劳动能力；
- 甲方患职业病以及现有医疗条件下难以治愈的严重疾病或者精神病；
- 甲方被判刑；
- 甲方违反劳动纪律、规章制度，给用人单位造成重大损害；
- 甲方正在接受纪律审查尚未作出结论；
- 甲方属于国家规定的不得解除本合同的其他情形；
- 乙方在试用期内；
- 乙方考入普通高等院校；
- 乙方被录用或者选调为公务员；
- 乙方依法服兵役。

除上述情形外，所有的变更和解除都需要甲乙双方协商一致。
